In [ ]:
import requests
# import random
# import polyline
import urllib3
import secrets
import pandas as pd
# import folium

In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Strava API Request

In [ ]:
auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'

print('Requesting Strava token... \n')
res = requests.post(auth_url, data=secrets.strava_payload, verify=False)
strava_access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + strava_access_token}

strava_requests_page_num = 1
all_activities = []

while True:
    strava_param = {'per_page' : 15, 'page' : strava_requests_page_num}
    strava_dataset = requests.get(activities_url, headers=header, params=strava_param).json()

    if len(strava_dataset) == 0:
        print('breaking out of Strava while loop because the response is zero, indicating no more activities.')
        break

    if all_activities:
        print('all activities is populated')
        all_activities.extend(strava_dataset)

    else:
        print('all activities is NOT populated')
        all_activities = strava_dataset

    strava_requests_page_num += 1

print('Total Activities: ', len(all_activities))

## Creating pandas dataframe for running data

In [ ]:
run_data = pd.DataFrame(data=all_activities)

In [ ]:
run_data.columns

In [ ]:
run_data

In [ ]:
columns_to_drop = ['athlete', 
                   'sport_type', 
                   'workout_type', 
                   'kudos_count', 
                   'comment_count', 
                   'athlete_count', 
                   'photo_count', 
                   'trainer', 
                   'commute', 
                   'manual', 
                   'private',
                   'visibility', 
                   'flagged', 
                   'gear_id', 
                   'has_heartrate', 
                   'heartrate_opt_out', 
                   'display_hide_heartrate_option', 
                   'from_accepted_tag', 
                   'total_photo_count', 
                   'has_kudoed', 
                   'average_watts', 
                   'kilojoules',
                   'achievement_count',
                   'device_watts', 
                   'suffer_score']

run_data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
run_data.columns

In [ ]:
map_data = pd.json_normalize(run_data['map'])
map_data_info = pd.DataFrame(map_data)
map_data_info

In [ ]:
run_data = pd.concat([run_data, map_data_info], axis=1)

In [ ]:
run_data

## Conversions for Metrics

In [ ]:
run_data['distance_miles'] = run_data['distance'] * 0.00062137119
run_data['moving_time_minutes'] = run_data['moving_time'] / 60
run_data['moving_time_hours'] = run_data['moving_time'] / 3600

In [ ]:
average_distance_miles = run_data['distance_miles'].mean()
print("Average Distance (miles):", average_distance_miles)

average_time_minutes = run_data['moving_time_minutes'].mean()
print("Average Time Ran (minutes):", average_time_minutes)

max_distance_ran = run_data['distance_miles'].max()
print("Longest Run:", max_distance_ran, "miles")

max_duration_mintues = run_data['moving_time_minutes'].max()
max_duration_hours = run_data['moving_time_hours'].max()
print("Longest Duration:", max_duration_mintues,"minutes. Converted to hours:", max_duration_hours)

total_distance_miles = run_data['distance'].sum() * 0.00062137119
print("Total Distance Covered to the date (miles):", total_distance_miles)